In [66]:
import cv2
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D, Dropout, Flatten
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from keras.callbacks import CSVLogger
from livelossplot import PlotLossesKeras
import joblib

In [67]:
train_path,train_dirs,train_files = next(os.walk("data/Train"))
labels = train_dirs
int_labels = [i for i in range(len(train_dirs))]

In [68]:
train_path,train_dirs,train_files = next(os.walk("Modified data/Train"))
test_path,test_dirs,test_files = next(os.walk("Modified data/Validation"))

In [72]:
train = pd.read_csv(train_path+"/$.csv")
test = pd.read_csv(test_path+"/$.csv")
temp_test = test
test = test.drop(labels="label",axis=1)
y = train["label"]
y = np.full((y.shape[0],),int(int_labels[0]))
x = train.drop(labels=["label"],axis=1)
#print(test)

In [73]:
#Reshaping
if(x.shape[1]):
    IMAGE_WIDTH = IMAGE_HEIGHT = 28
else:
    IMAGE_WIDTH = IMAGE_HEIGHT = 32
CHANNELS = 1
CLASSES = 39
VALIDATION_RATIO = 0.7
TRAINING_LOGS_FILE = "/Logs"
EPOCHS=1
BATCH_SIZE=32
#VERBOSITY=1

In [74]:
x = x.values.reshape(-1, IMAGE_WIDTH, IMAGE_HEIGHT, CHANNELS)
test = test.values.reshape(-1, IMAGE_WIDTH, IMAGE_HEIGHT, CHANNELS)

y = to_categorical(y, num_classes=CLASSES)

In [75]:
#Data Splitting
x_training, x_validation, y_training, y_validation = train_test_split(x, y, test_size=VALIDATION_RATIO)

In [76]:
#Data Augmentation
data_generator = ImageDataGenerator(rescale=1./255,
                                    rotation_range=10,
                                    zoom_range=0.15,
                                    width_shift_range=0.1,
                                    height_shift_range=0.1)
data_generator.fit(x_training)

In [14]:
#Building the model
model = Sequential()

In [15]:
model.add(Conv2D(filters=32,
                 kernel_size=(5,5),
                 padding='Same',
                 activation='relu',
                 input_shape=(IMAGE_WIDTH, IMAGE_HEIGHT, CHANNELS)))

In [16]:
model.add(Conv2D(filters=32,
                 kernel_size=(5,5),
                 padding='Same',
                 activation='relu'))

In [17]:
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.5))

In [18]:
model.add(Conv2D(filters=64,
                 kernel_size=(3,3),
                 padding='Same',
                 activation='relu'))

In [19]:
model.add(Conv2D(filters=64,
                 kernel_size=(3,3),
                 padding='Same',
                 activation='relu'))

In [20]:
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.5))

In [21]:
model.add(Flatten())

In [22]:
model.add(Dense(8192, activation='relu'))
model.add(Dropout(0.5))

In [23]:
model.add(Dense(2048, activation='relu'))
model.add(Dropout(0.5))

In [24]:
model.add(Dense(CLASSES, activation="softmax"))

In [33]:
model.compile(optimizer=RMSprop(lr=0.0001,
                                rho=0.9,
                                epsilon=1e-8,
                                decay=0.00001),
             loss="categorical_crossentropy",
             metrics=["accuracy"])

In [ ]:
#Training
history = model.fit_generator(data_generator.flow(x_training,
                                                  y_training,
                                                  batch_size=BATCH_SIZE),
                              epochs=EPOCHS,
                              validation_data=(x_validation, y_validation),
                              #verbose=VERBOSITY,
                              steps_per_epoch=x_training.shape[0],
                              callbacks=[PlotLossesKeras(),
                                         CSVLogger(TRAINING_LOGS_FILE,
                                         append=False,
                                         separator=";")])

Epoch 1/1
2468/4859 [==============>...............] - ETA: 41:41 - loss: 0.0000e+00 - accuracy: 1.0000

In [53]:
#Testing
predictions = model.predict_classes(test, verbose=1)
pd.DataFrame({"ImageId":list(range(1,len(predictions)+1)),
            "Label":predictions}).to_csv("&outputfile.csv")

1400/1400 [==============================] - 4s 3ms/step


In [54]:
filename = "#model.exe"
joblib.dump(model,filename)

['#model.exe']